In [1]:
import pandas as pd

# Load the main dataset
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')

X.set_index('Primary ID', inplace=True)
y.set_index('Primary ID', inplace=True)

# Load the column names from the Excel files (assuming they are in the first row)
padel_cols = pd.read_excel('Padel_cols.xlsx', header=None).iloc[0].dropna().tolist()
spartan_cols = pd.read_excel('Spartan_cols.xlsx', header=None).iloc[0].dropna().tolist()
swissadme_cols = pd.read_excel('Swissadme_cols.xlsx', header=None).iloc[0].dropna().tolist()


Check non-numeric data

In [2]:
non_numeric_columns = X.select_dtypes(exclude=['number']).columns
non_numeric_columns

Index([], dtype='object')

Check rows with nans

In [3]:
def check_nans(data):
    rows_with_nans = data.isnull().any(axis=1)
    num_rows_with_nans = rows_with_nans.sum()
    total_rows = len(data)
    fraction_rows_with_nans = num_rows_with_nans / total_rows

    print(f"Number of rows with NaNs: {num_rows_with_nans}")
    print(f"Fraction of rows with NaNs: {fraction_rows_with_nans:.2f}")

    # Identify columns with NaNs
    columns_with_nans = data.columns[data.isnull().any()].tolist()
    num_columns_with_nans = len(columns_with_nans)

    print(f"Columns with NaNs: {columns_with_nans}")

    # Print detailed information about NaNs in each column
    nan_info = data.isnull().sum()
    print("\nDetailed NaN information:")
    print(nan_info[nan_info > 0])
    print("Number of columns with nan values:")
    print(f"{num_columns_with_nans} columns out of the total {len(data.columns)} columns")

In [4]:
from sklearn.decomposition import PCA
import numpy as np
import joblib

def perform_pca(X_subset, subset_name, explained_variance_threshold=0.95):
    # Print data nans before standardization
    print("NANs before standardization")
    check_nans(X_subset)

    # Drop zero variance columns
    zero_variance_columns = X_subset.loc[:, X_subset.std() == 0].columns
    X_subset = X_subset.drop(columns=zero_variance_columns)
    # save the columns to drop
    joblib.dump(zero_variance_columns, subset_name + '_cols_to_drop.pkl')
    
    # Standardize the data if necessary
    mean = X_subset.mean()
    std = X_subset.std()
    X_standardized = (X_subset - mean) / std
    # save the values for standardization
    joblib.dump(mean, subset_name + '_mean.pkl')
    joblib.dump(std, subset_name + '_std.pkl')

    # Print data nans before standardization
    print("NANs after standardization")
    check_nans(X_standardized)

    # Initialize PCA
    pca = PCA()

    # Fit PCA
    pca.fit(X_standardized)

    # Calculate cumulative explained variance
    cum_var_explained = np.cumsum(pca.explained_variance_ratio_)
    
    # Determine the number of components needed to reach the explained variance threshold
    num_components = np.argmax(cum_var_explained >= explained_variance_threshold) + 1
    
    # Apply PCA with the selected number of components
    pca = PCA(n_components=num_components)
    X_pca = pca.fit_transform(X_standardized)

    # Save the scaler
    joblib.dump(pca, subset_name + '_pca_fit.pkl')
    
    return X_pca, num_components, pca.explained_variance_ratio_

# Perform PCA on each set of columns
X_padel_pca, padel_n_components, padel_variance_ratio = perform_pca(X[padel_cols], subset_name='padel')
X_spartan_pca, spartan_n_components, spartan_variance_ratio = perform_pca(X[spartan_cols], subset_name='spartan')
X_swissadme_pca, swissadme_n_components, swissadme_variance_ratio = perform_pca(X[swissadme_cols], subset_name='swissadme')

# Display the number of components retained
print(f"Padel: {padel_n_components} components retained")
print(f"Spartan: {spartan_n_components} components retained")
print(f"SwissADME: {swissadme_n_components} components retained")


NANs before standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 1444 columns
NANs after standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 1083 columns
NANs before standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 23 columns
NANs after standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 23 columns
NANs before standardization
Number of rows with NaNs: 

Remove NaN rows

After performing PCA, merge the datasets back together

In [5]:
# Convert PCA results to DataFrames with 'Primary ID' as the index
X_padel_pca_df = pd.DataFrame(X_padel_pca, index=X.index)
X_spartan_pca_df = pd.DataFrame(X_spartan_pca, index=X.index)
X_swissadme_pca_df = pd.DataFrame(X_swissadme_pca, index=X.index)

# Ensure that the other features DataFrame is also indexed by 'Primary ID'
other_features_cols = ["API dose", "API %", "Plast %"]
X_other_features = X[other_features_cols].set_index(X.index)
print(f"Other features: {X_other_features}")

# Normalize the rest of the columns
mean = X_other_features.mean()
std = X_other_features.std()
X_standardized = (X_other_features - mean) / std
# save the values for standardization
joblib.dump(mean, 'non_pca_features_mean.pkl')
joblib.dump(std, 'non_pca_features_std.pkl')

# Merge the PCA-transformed data back together with the rest of the features
X_final = pd.concat([X_padel_pca_df, X_spartan_pca_df, X_swissadme_pca_df, X_standardized], axis=1)

# Make sure all the column names are strings
X_final.columns = X_final.columns.astype(str)

# Verify the final DataFrame shape and columns
print(f"Final DataFrame shape: {X_final.shape}")
print(X_final)

Other features:                  API dose  API %  Plast %
Primary ID                               
Amlodipine_N1_1      7.90      5      0.0
Amlodipine_N1_2      9.30      5      0.0
Amlodipine_N2_1     23.40     15      0.0
Amlodipine_N2_2     22.05     15      0.0
Amlodipine_N4_1      8.25      5      7.5
...                   ...    ...      ...
Diclofenac_N2_2     25.95     15      0.0
Diclofenac_N4_1      7.55      5      7.5
Diclofenac_N4_2      8.10      5      7.5
Diclofenac_N5_1     24.30     15      7.5
Diclofenac_N5_2     23.40     15      7.5

[220 rows x 3 columns]
Final DataFrame shape: (220, 25)
                         0         1         2         3          4         5  \
Primary ID                                                                      
Amlodipine_N1_1  20.953686  5.689311 -6.191519  0.595709  -3.476437 -9.792322   
Amlodipine_N1_2  20.953686  5.689311 -6.191519  0.595709  -3.476437 -9.792322   
Amlodipine_N2_1  20.953686  5.689311 -6.191519  0.595709 

Check for duplicates in the column names between the 3 data sources (Padel, Spartan, SwissADME)

In [6]:
X_final.index

Index(['Amlodipine_N1_1', 'Amlodipine_N1_2', 'Amlodipine_N2_1',
       'Amlodipine_N2_2', 'Amlodipine_N4_1', 'Amlodipine_N4_2',
       'Amlodipine_N5_1', 'Amlodipine_N5_2', 'Amlodipine_N10_1',
       'Amlodipine_N10_2',
       ...
       'Atenolol_N10_1', 'Atenolol_N10_2', 'Atenolol_N11_1', 'Atenolol_N11_2',
       'Diclofenac_N2_1', 'Diclofenac_N2_2', 'Diclofenac_N4_1',
       'Diclofenac_N4_2', 'Diclofenac_N5_1', 'Diclofenac_N5_2'],
      dtype='object', name='Primary ID', length=220)

In [7]:
X_final.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '0', '1', '2', '3', '4',
       '5', '0', '1', '2', '3', '4', '5', '6', 'API dose', 'API %', 'Plast %'],
      dtype='object')

In [8]:
y.index

Index(['Amlodipine_N1_1', 'Amlodipine_N1_2', 'Amlodipine_N2_1',
       'Amlodipine_N2_2', 'Amlodipine_N4_1', 'Amlodipine_N4_2',
       'Amlodipine_N5_1', 'Amlodipine_N5_2', 'Amlodipine_N10_1',
       'Amlodipine_N10_2',
       ...
       'Atenolol_N10_1', 'Atenolol_N10_2', 'Atenolol_N11_1', 'Atenolol_N11_2',
       'Diclofenac_N2_1', 'Diclofenac_N2_2', 'Diclofenac_N4_1',
       'Diclofenac_N4_2', 'Diclofenac_N5_1', 'Diclofenac_N5_2'],
      dtype='object', name='Primary ID', length=220)

In [9]:
y.columns

Index(['%5min', '%10min', '%15min', '%30min', '%60min', '%120min', '%180min',
       '%240min'],
      dtype='object')

In [10]:
X_final.to_csv("X_PCA.csv")
y.to_csv("y_PCA.csv")

In [11]:
X_final

,0,1,2,3,4,5,6,7,8,0,...,0,1,2,3,4,5,6,API dose,API %,Plast %
Primary ID,,,,,,,,,,,,,,,,,,,,,
Amlodipine_N1_1,20.953686,5.689311,-6.191519,0.595709,-3.476437,-9.792322,2.797484,7.901833,1.542751,3.644278,...,3.903647,1.126563,1.340683,0.709167,0.131362,0.302066,-0.234016,-1.328160,-1.325665,-1.326091
Amlodipine_N1_2,20.953686,5.689311,-6.191519,0.595709,-3.476437,-9.792322,2.797484,7.901833,1.542751,3.644278,...,3.903647,1.126563,1.340683,0.709167,0.131362,0.302066,-0.234016,-1.212977,-1.325665,-1.326091
Amlodipine_N2_1,20.953686,5.689311,-6.191519,0.595709,-3.476437,-9.792322,2.797484,7.901833,1.542751,3.644278,...,3.903647,1.126563,1.340683,0.709167,0.131362,0.302066,-0.234016,-0.052920,0.063127,-1.326091
Amlodipine_N2_2,20.953686,5.689311,-6.191519,0.595709,-3.476437,-9.792322,2.797484,7.901833,1.542751,3.644278,...,3.903647,1.126563,1.340683,0.709167,0.131362,0.302066,-0.234016,-0.163989,0.063127,-1.326091
Amlodipine_N4_1,20.953686,5.689311,-6.191519,0.595709,-3.476437,-9.792322,2.797484,7.901833,1.542751,3.644278,...,3.903647,1.126563,1.340683,0.709167,0.131362,0.302066,-0.234016,-1.299364,-1.325665,0.037180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diclofenac_N2_2,-0.501396,1.398084,-1.749690,8.410942,-11.487461,4.203837,-1.636791,-1.416689,9.206061,-0.053670,...,4.032256,-1.942994,-1.594053,-4.712624,1.394431,-2.843797,3.847974,0.156877,0.063127,-1.326091
Diclofenac_N4_1,-0.501396,1.398084,-1.749690,8.410942,-11.487461,4.203837,-1.636791,-1.416689,9.206061,-0.053670,...,4.032256,-1.942994,-1.594053,-4.712624,1.394431,-2.843797,3.847974,-1.356956,-1.325665,0.037180
Diclofenac_N4_2,-0.501396,1.398084,-1.749690,8.410942,-11.487461,4.203837,-1.636791,-1.416689,9.206061,-0.053670,...,4.032256,-1.942994,-1.594053,-4.712624,1.394431,-2.843797,3.847974,-1.311705,-1.325665,0.037180


# Train Random Forest Multiregressor

In [12]:
# test-train split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

# Initialize the base model
base_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create a multi-output regressor
multi_output_model = MultiOutputRegressor(base_model)

# Train the model
multi_output_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

Evaluate the model

In [15]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the test set
y_pred = multi_output_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='variance_weighted')

# Print MSE and target variable names
for target_name, mse_value in zip(y_test.columns, mse):
    print(f"Target: {target_name}, Mean Squared Error: {mse_value}")

print(f"\nOverall R^2 Score: {r2}")


Target: %5min, Mean Squared Error: 3.9591139240083
Target: %10min, Mean Squared Error: 6.333494555070928
Target: %15min, Mean Squared Error: 11.455237859469207
Target: %30min, Mean Squared Error: 28.68573916236111
Target: %60min, Mean Squared Error: 130.71487930189252
Target: %120min, Mean Squared Error: 82.29820582931023
Target: %180min, Mean Squared Error: 48.37663891029547
Target: %240min, Mean Squared Error: 22.837551575557594

Overall R^2 Score: 0.6284248654085796


In [16]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Predict on the test set
y_pred = multi_output_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')  # MSE for each target
r2_per_target = r2_score(y_test, y_pred, multioutput='raw_values')  # R² for each target
r2_overall = r2_score(y_test, y_pred, multioutput='variance_weighted')  # Overall weighted R²

# Calculate variance or range of each target for relative performance
variance_targets = np.var(y_test, axis=0)  # Variance of each target in test set
range_targets = np.ptp(y_test, axis=0)  # Range (max-min) of each target in test set

# Print MSE, R² score, variance, and range for each target
for target_name, mse_value, r2_value, variance_value, range_value in zip(y_test.columns, mse, r2_per_target, variance_targets, range_targets):
    print(f"Target: {target_name}, Mean Squared Error: {mse_value}, R² Score: {r2_value}")
    print(f"Target: {target_name}, Variance: {variance_value}, Range: {range_value}")
    print(f"Relative MSE (MSE/Variance): {mse_value/variance_value if variance_value != 0 else 'Undefined'}")
    print(f"Relative MSE (MSE/Range): {mse_value/range_value if range_value != 0 else 'Undefined'}\n")

# Print overall R² score
print(f"\nOverall R^2 Score: {r2_overall}")


Target: %5min, Mean Squared Error: 3.9591139240083, R² Score: 0.6834025028712658
Target: %5min, Variance: 12.505196534761149, Range: 14.584043115914138
Relative MSE (MSE/Variance): 0.3165974971287342
Relative MSE (MSE/Range): 0.27146888503696937

Target: %10min, Mean Squared Error: 6.333494555070928, R² Score: 0.6701711180743146
Target: %10min, Variance: 19.202364929636285, Range: 18.424010309873367
Relative MSE (MSE/Variance): 0.3298288819256854
Relative MSE (MSE/Range): 0.34376308135676786

Target: %15min, Mean Squared Error: 11.455237859469207, R² Score: 0.6700363066756001
Target: %15min, Variance: 34.716661533446725, Range: 29.364562625630754
Relative MSE (MSE/Variance): 0.32996369332439995
Relative MSE (MSE/Range): 0.39010415395973047

Target: %30min, Mean Squared Error: 28.68573916236111, R² Score: 0.6304648477219263
Target: %30min, Variance: 77.6265505068249, Range: 45.235241812542064
Relative MSE (MSE/Variance): 0.3695351522780736
Relative MSE (MSE/Range): 0.6341458122681597

T

In [17]:
import joblib

# Save the model
joblib.dump(multi_output_model, 'multi_output_model.pkl')

['multi_output_model.pkl']